In [ ]:
#Working

import fitz
import tkinter as tk
from tkinter import filedialog, messagebox
import io

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=20)

        self.redact_button = tk.Button(self.window, text="Redact Selected Areas", command=self.redact)
        self.redact_button.pack(pady=20)
        
        self.selected_rectangles = []

        self.window.geometry("300x200")
        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.display_pdf()
            
    def display_pdf(self):
        self.current_page = 0
        self.page_image = self.pdf_document[self.current_page].get_pixmap()
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title("PDF Viewer") 
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()
        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

    def on_button_press(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_button_release(self, event):
        end_x = event.x
        end_y = event.y
        self.selected_rectangles.append((self.start_x, self.start_y, end_x, end_y))
        self.canvas.create_rectangle(self.start_x, self.start_y, end_x, end_y, outline="red")

    def redact(self):
        output_pdf = "redacted_output.pdf"
        for rect in self.selected_rectangles:
            self.redact_area(rect)
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def redact_area(self, rect):
        for page in self.pdf_document:
            rect_fitz = fitz.Rect(rect)
            page.add_redact_annot(rect_fitz)
            page.apply_redactions()

if __name__ == "__main__":
    PDFRedactor()
